#### Installation

Install the following packages required to execute this notebook.

In [24]:
! pip3 install --user --no-cache-dir --upgrade "kfp>2" "google-cloud-pipeline-components>2"\
                                        google-cloud-aiplatform

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.4/372.4 kB 10.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for kfp: filename=kfp-2.1.3-py3-none-any.whl size=521300 sha256=576b928df862fb050cb83122e75c741a2ef824a575042cab910d159c3c6b2db9
  Stored in directory: /tmp/pip-ephem-wheel-cache-291hyrrd/wheels/21/bf/76/46fcbf67e695f5c2f2159e940cb7594e2845f475061fd44223
Successfully built kfp
  Attempting uninstall: kfp
    Found existing installation: kfp 2.3.0
    Uninstalling kfp-2.3.0:
      Successfully uninstalled kfp-2.3.0
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


### Check the KFP SDK version.

In [25]:
! python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"
! pip3 freeze | grep aiplatform

KFP SDK version: 2.1.3
google-cloud-aiplatform==1.33.1


#### Pipeline Configurations

In [1]:
#The Google Cloud project that this pipeline runs in.
PROJECT_ID = "your_project_id"
# The region that this pipeline runs in
REGION = "us-central1"
# Specify a Cloud Storage URI that your pipelines service account can access. The artifacts of your pipeline runs are stored within the pipeline root.
PIPELINE_ROOT = "gs://your_temp_bucket"

In [ ]:
#### Import Packages

In [27]:
import google.cloud.aiplatform as aiplatform
import kfp
from kfp import compiler, dsl
from kfp.dsl import Artifact, Dataset, Input, Metrics, Model, Output, component

In [28]:
# Initialize the AI platform
aiplatform.init(
    project=PROJECT_ID,
    location=REGION,
)

#### Create Pipeline Components

We can create a component from Python functions (inline) and from a container. We will first try inline python functions. 
Refer to  https://www.kubeflow.org/docs/components/pipelines/v2/components/lightweight-python-components/ for more information.

#### Pipeline Component : Add

In [29]:
@dsl.component
def add(a: float, b: float) -> float:
  '''Calculates sum of two arguments'''
  return a + b

In [30]:
@dsl.pipeline(
  name='addition-pipeline',
  description='An example pipeline that performs addition calculations.',
  pipeline_root=PIPELINE_ROOT
)
def add_pipeline(
  x: float=1.0,
  y: float=7.0,
):
  # Passes a pipeline parameter and a constant value to the `add` factory
  # function.
  first_add_task = add(a=x, b=4.0)
  # Passes an output reference from `first_add_task` and a pipeline parameter
  # to the `add` factory function. For operations with a single return
  # value, the output reference can be accessed as `task.output` or
  # `task.outputs['output_name']`.
  second_add_task = add(a=first_add_task.output, b=y)

#### Compile the pipeline into a JSON file

In [31]:
from kfp import compiler

compiler.Compiler().compile(
    pipeline_func=add_pipeline,
    package_path='add_pipeline.yaml'
)

#### Submit the pipeline run

In [32]:
import google.cloud.aiplatform as aip


# Prepare the pipeline job
job = aip.PipelineJob(
    display_name="add_pipeline",
    template_path="add_pipeline.yaml",
    pipeline_root=PIPELINE_ROOT,
    parameter_values={
        'x': 8.0,
        'y': 9.0
    }
)

job.run()